In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.metrics import mean_squared_error
from math import sqrt

In [61]:
train = pd.read_csv("../data/schemas/warm_up/TrainSet.csv", sep=",")
validation = pd.read_csv("../data/schemas/warm_up/ValidationSet.csv", sep=",")
test = pd.read_csv("../data/schemas/warm_up/TestSet.csv", sep=",")

In [62]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.4   # For feature selection

In [63]:
columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[0]])) > THRESHOLD: #or abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
independent_variables.remove(columns_to_predict[0])
    
if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
print(independent_variables)

['Hour_1', 'Hour_2', 'Hour_3', 'Occupant Count (people)_3', '12h Outdoor Drybulb Temperature (C)', '24h Outdoor Drybulb Temperature (C)', '12h Outdoor Relative Humidity (%)', '6h Direct Solar Radiation (W/m2)', '12h Direct Solar Radiation (W/m2)']


Prepare the data

In [64]:
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

print(X_train_default.shape)
print(X_val_default.shape)
print(X_test_default.shape)

(576, 9)
(96, 9)
(96, 9)


In [65]:
# Function to prepare the data into batches that will be passed into the model

def create_sequences(input_data, output_data, timesteps_input, timesteps_output):
    sequences, targets = [], []
    for i in range(len(input_data) - timesteps_input - timesteps_output + 1):
        seq = input_data[i:i + timesteps_input]
        target = output_data[i + timesteps_input: i + timesteps_input + timesteps_output]
        sequences.append(seq)
        targets.append(target)

    return np.array(sequences), np.array(targets)

In [66]:
X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

print(f"X_train = {X_train.shape}, Y_train = {Y_train.shape}\n"
      f"X_val = {X_val.shape}, Y_val = {Y_val.shape}\n"
      f"X_test = {X_test.shape}, Y_test = {Y_test.shape}")

X_train = (505, 48, 9), Y_train = (505, 24, 2)
X_val = (25, 48, 9), Y_val = (25, 24, 2)
X_test = (25, 48, 9), Y_test = (25, 24, 2)


In [67]:
print(Y_train[:, :, 0])

[[0.59739428 0.49429981 0.38708765 ... 0.73547528 0.69513525 0.71729728]
 [0.49429981 0.38708765 0.28880037 ... 0.69513525 0.71729728 0.6633147 ]
 [0.38708765 0.28880037 0.2346659  ... 0.71729728 0.6633147  0.55539687]
 ...
 [0.47416969 0.39741721 0.28594384 ... 0.4397153  0.45619247 0.45482902]
 [0.39741721 0.28594384 0.16805223 ... 0.45619247 0.45482902 0.41836388]
 [0.28594384 0.16805223 0.12974031 ... 0.45482902 0.41836388 0.39162712]]


In [68]:
RMSE_model_1 = []
RMSE_model_2 = []

In [69]:
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras import layers, models

In [70]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(120, activation='tanh')(flatten_layer)
dense2 = layers.Dense(60, activation='tanh')(dense1)
dense3 = layers.Dense(24, activation='tanh')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(120, activation='tanh')(flatten_layer)
dense5 = layers.Dense(60, activation='tanh')(dense4)
dense6 = layers.Dense(24, activation='tanh')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_test)):
    current_batch = X_test[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 2s 55ms/step - loss: 0.7987 - output_variable1_loss: 0.4613 - output_variable2_loss: 0.3374 - val_loss: 0.3930 - val_output_variable1_loss: 0.1853 - val_output_variable2_loss: 0.2077
Epoch 2/100
4/4 [==============================] - 0s 11ms/step - loss: 0.3973 - output_variable1_loss: 0.2331 - output_variable2_loss: 0.1642 - val_loss: 0.2765 - val_output_variable1_loss: 0.1313 - val_output_variable2_loss: 0.1453
Epoch 3/100
4/4 [==============================] - 0s 11ms/step - loss: 0.2884 - output_variable1_loss: 0.1654 - output_variable2_loss: 0.1229 - val_loss: 0.2013 - val_output_variable1_loss: 0.0935 - val_output_variable2_loss: 0.1078
Epoch 4/100
4/4 [==============================] - 0s 10ms/step - loss: 0.2253 - output_variable1_loss: 0.1250 - output_variable2_loss: 0.1003 - val_loss: 0.1668 - val_output_variable1_loss: 0.0785 - val_output_variable2_loss: 0.0883
Epoch 5/100
4/4 [==============================] - 0s 11ms/step 

In [71]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(120, activation='relu')(flatten_layer)
dense2 = layers.Dense(60, activation='relu')(dense1)
dense3 = layers.Dense(24, activation='relu')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(120, activation='relu')(flatten_layer)
dense5 = layers.Dense(60, activation='relu')(dense4)
dense6 = layers.Dense(24, activation='relu')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_test)):
    current_batch = X_test[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 1s 54ms/step - loss: 0.7241 - output_variable1_loss: 0.4280 - output_variable2_loss: 0.2960 - val_loss: 0.3703 - val_output_variable1_loss: 0.1610 - val_output_variable2_loss: 0.2093
Epoch 2/100
4/4 [==============================] - 0s 11ms/step - loss: 0.4522 - output_variable1_loss: 0.2689 - output_variable2_loss: 0.1833 - val_loss: 0.3366 - val_output_variable1_loss: 0.1530 - val_output_variable2_loss: 0.1836
Epoch 3/100
4/4 [==============================] - 0s 10ms/step - loss: 0.3772 - output_variable1_loss: 0.2151 - output_variable2_loss: 0.1621 - val_loss: 0.2852 - val_output_variable1_loss: 0.1253 - val_output_variable2_loss: 0.1599
Epoch 4/100
4/4 [==============================] - 0s 11ms/step - loss: 0.3168 - output_variable1_loss: 0.1741 - output_variable2_loss: 0.1427 - val_loss: 0.2555 - val_output_variable1_loss: 0.1152 - val_output_variable2_loss: 0.1403
Epoch 5/100
4/4 [==============================] - 0s 11ms/step 

In [72]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(240, activation='relu')(flatten_layer)
dense2 = layers.Dense(120, activation='relu')(dense1)
dense3 = layers.Dense(48, activation='relu')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(240, activation='relu')(flatten_layer)
dense5 = layers.Dense(120, activation='relu')(dense4)
dense6 = layers.Dense(48, activation='relu')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_test)):
    current_batch = X_test[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 1s 56ms/step - loss: 0.6863 - output_variable1_loss: 0.4770 - output_variable2_loss: 0.2094 - val_loss: 0.3116 - val_output_variable1_loss: 0.1678 - val_output_variable2_loss: 0.1438
Epoch 2/100
4/4 [==============================] - 0s 12ms/step - loss: 0.3584 - output_variable1_loss: 0.2306 - output_variable2_loss: 0.1278 - val_loss: 0.2640 - val_output_variable1_loss: 0.1517 - val_output_variable2_loss: 0.1123
Epoch 3/100
4/4 [==============================] - 0s 12ms/step - loss: 0.2737 - output_variable1_loss: 0.1750 - output_variable2_loss: 0.0986 - val_loss: 0.1962 - val_output_variable1_loss: 0.1087 - val_output_variable2_loss: 0.0875
Epoch 4/100
4/4 [==============================] - 0s 12ms/step - loss: 0.2229 - output_variable1_loss: 0.1433 - output_variable2_loss: 0.0796 - val_loss: 0.1782 - val_output_variable1_loss: 0.1019 - val_output_variable2_loss: 0.0763
Epoch 5/100
4/4 [==============================] - 0s 12ms/step 

In [73]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(480, activation='relu')(flatten_layer)
dense2 = layers.Dense(240, activation='relu')(dense1)
dense3 = layers.Dense(120, activation='relu')(dense2)
dense9 = layers.Dense(48, activation='relu')(dense3)
output_variable1 = layers.Dense(24, name='output_variable1')(dense9)

# Dense layers for variable 2
dense4 = layers.Dense(480, activation='relu')(flatten_layer)
dense5 = layers.Dense(240, activation='relu')(dense4)
dense6 = layers.Dense(120, activation='relu')(dense5)
dense7 = layers.Dense(48, activation='relu')(dense6)
output_variable2 = layers.Dense(24, name='output_variable2')(dense7)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=200, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_test)):
    current_batch = X_test[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/200
4/4 [==============================] - 1s 61ms/step - loss: 0.4651 - output_variable1_loss: 0.2748 - output_variable2_loss: 0.1904 - val_loss: 0.2557 - val_output_variable1_loss: 0.1134 - val_output_variable2_loss: 0.1423
Epoch 2/200
4/4 [==============================] - 0s 14ms/step - loss: 0.2205 - output_variable1_loss: 0.0964 - output_variable2_loss: 0.1241 - val_loss: 0.1942 - val_output_variable1_loss: 0.0772 - val_output_variable2_loss: 0.1170
Epoch 3/200
4/4 [==============================] - 0s 14ms/step - loss: 0.1577 - output_variable1_loss: 0.0552 - output_variable2_loss: 0.1026 - val_loss: 0.1515 - val_output_variable1_loss: 0.0555 - val_output_variable2_loss: 0.0960
Epoch 4/200
4/4 [==============================] - 0s 14ms/step - loss: 0.1250 - output_variable1_loss: 0.0396 - output_variable2_loss: 0.0854 - val_loss: 0.1465 - val_output_variable1_loss: 0.0651 - val_output_variable2_loss: 0.0814
Epoch 5/200
4/4 [==============================] - 0s 14ms/step 

In [74]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(60, activation='relu')(flatten_layer)
dense2 = layers.Dense(48, activation='relu')(dense1)
dense3 = layers.Dense(24, activation='relu')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(60, activation='relu')(flatten_layer)
dense5 = layers.Dense(48, activation='relu')(dense4)
dense6 = layers.Dense(24, activation='relu')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_test)):
    current_batch = X_test[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 1s 56ms/step - loss: 0.6502 - output_variable1_loss: 0.4381 - output_variable2_loss: 0.2121 - val_loss: 0.4001 - val_output_variable1_loss: 0.2002 - val_output_variable2_loss: 0.1999
Epoch 2/100
4/4 [==============================] - 0s 11ms/step - loss: 0.5049 - output_variable1_loss: 0.3274 - output_variable2_loss: 0.1775 - val_loss: 0.3670 - val_output_variable1_loss: 0.1883 - val_output_variable2_loss: 0.1787
Epoch 3/100
4/4 [==============================] - 0s 11ms/step - loss: 0.4428 - output_variable1_loss: 0.2840 - output_variable2_loss: 0.1589 - val_loss: 0.3192 - val_output_variable1_loss: 0.1591 - val_output_variable2_loss: 0.1600
Epoch 4/100
4/4 [==============================] - 0s 11ms/step - loss: 0.3860 - output_variable1_loss: 0.2425 - output_variable2_loss: 0.1434 - val_loss: 0.2919 - val_output_variable1_loss: 0.1477 - val_output_variable2_loss: 0.1441
Epoch 5/100
4/4 [==============================] - 0s 11ms/step 

In [75]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 8) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(120, activation='relu')(flatten_layer)
dense2 = layers.Dense(60, activation='relu')(dense1)
dense3 = layers.Dense(24, activation='relu')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(120, activation='relu')(flatten_layer)
dense5 = layers.Dense(60, activation='relu')(dense4)
dense6 = layers.Dense(24, activation='relu')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_test)):
    current_batch = X_test[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 1s 54ms/step - loss: 0.6432 - output_variable1_loss: 0.4288 - output_variable2_loss: 0.2143 - val_loss: 0.3580 - val_output_variable1_loss: 0.1987 - val_output_variable2_loss: 0.1593
Epoch 2/100
4/4 [==============================] - 0s 11ms/step - loss: 0.4702 - output_variable1_loss: 0.3322 - output_variable2_loss: 0.1381 - val_loss: 0.3183 - val_output_variable1_loss: 0.1845 - val_output_variable2_loss: 0.1338
Epoch 3/100
4/4 [==============================] - 0s 11ms/step - loss: 0.4103 - output_variable1_loss: 0.2934 - output_variable2_loss: 0.1169 - val_loss: 0.2801 - val_output_variable1_loss: 0.1624 - val_output_variable2_loss: 0.1177
Epoch 4/100
4/4 [==============================] - 0s 11ms/step - loss: 0.3632 - output_variable1_loss: 0.2587 - output_variable2_loss: 0.1045 - val_loss: 0.2638 - val_output_variable1_loss: 0.1581 - val_output_variable2_loss: 0.1058
Epoch 5/100
4/4 [==============================] - 0s 11ms/step 

In [76]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 8) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(120, activation='relu')(flatten_layer)
dense2 = layers.Dense(60, activation='relu')(dense1)
dense3 = layers.Dense(24, activation='relu')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(120, activation='relu')(flatten_layer)
dense5 = layers.Dense(60, activation='relu')(dense4)
dense6 = layers.Dense(24, activation='relu')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=200, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_test)):
    current_batch = X_test[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/200
4/4 [==============================] - 1s 60ms/step - loss: 0.6494 - output_variable1_loss: 0.3772 - output_variable2_loss: 0.2721 - val_loss: 0.4369 - val_output_variable1_loss: 0.2153 - val_output_variable2_loss: 0.2216
Epoch 2/200
4/4 [==============================] - 0s 13ms/step - loss: 0.5022 - output_variable1_loss: 0.3029 - output_variable2_loss: 0.1993 - val_loss: 0.3732 - val_output_variable1_loss: 0.1724 - val_output_variable2_loss: 0.2008
Epoch 3/200
4/4 [==============================] - 0s 14ms/step - loss: 0.4615 - output_variable1_loss: 0.2795 - output_variable2_loss: 0.1820 - val_loss: 0.3546 - val_output_variable1_loss: 0.1668 - val_output_variable2_loss: 0.1878
Epoch 4/200
4/4 [==============================] - 0s 13ms/step - loss: 0.4248 - output_variable1_loss: 0.2543 - output_variable2_loss: 0.1706 - val_loss: 0.3362 - val_output_variable1_loss: 0.1568 - val_output_variable2_loss: 0.1794
Epoch 5/200
4/4 [==============================] - 0s 12ms/step 

In [77]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(96, activation='relu')(flatten_layer)
dense2 = layers.Dense(48, activation='relu')(dense1)
output_variable1 = layers.Dense(24, name='output_variable1')(dense2)

# Dense layers for variable 2
dense4 = layers.Dense(96, activation='relu')(flatten_layer)
dense5 = layers.Dense(48, activation='relu')(dense4)
output_variable2 = layers.Dense(24, name='output_variable2')(dense5)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_test)):
    current_batch = X_test[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 1s 51ms/step - loss: 1.0251 - output_variable1_loss: 0.5525 - output_variable2_loss: 0.4726 - val_loss: 0.3817 - val_output_variable1_loss: 0.1725 - val_output_variable2_loss: 0.2092
Epoch 2/100
4/4 [==============================] - 0s 10ms/step - loss: 0.4450 - output_variable1_loss: 0.2659 - output_variable2_loss: 0.1791 - val_loss: 0.3294 - val_output_variable1_loss: 0.1665 - val_output_variable2_loss: 0.1630
Epoch 3/100
4/4 [==============================] - 0s 11ms/step - loss: 0.3635 - output_variable1_loss: 0.2179 - output_variable2_loss: 0.1457 - val_loss: 0.3002 - val_output_variable1_loss: 0.1652 - val_output_variable2_loss: 0.1351
Epoch 4/100
4/4 [==============================] - 0s 10ms/step - loss: 0.3060 - output_variable1_loss: 0.1842 - output_variable2_loss: 0.1218 - val_loss: 0.2368 - val_output_variable1_loss: 0.1238 - val_output_variable2_loss: 0.1130
Epoch 5/100
4/4 [==============================] - 0s 10ms/step 

In [78]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.5  # For feature selection
columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[
        0]])) > THRESHOLD:  #or abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
independent_variables.remove(columns_to_predict[0])

if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(120, activation='tanh')(flatten_layer)
dense2 = layers.Dense(60, activation='tanh')(dense1)
dense3 = layers.Dense(24, activation='tanh')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(120, activation='tanh')(flatten_layer)
dense5 = layers.Dense(60, activation='tanh')(dense4)
dense6 = layers.Dense(24, activation='tanh')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_test)):
    current_batch = X_test[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

print(independent_variables)

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 1s 55ms/step - loss: 0.7809 - output_variable1_loss: 0.5040 - output_variable2_loss: 0.2769 - val_loss: 0.3443 - val_output_variable1_loss: 0.1689 - val_output_variable2_loss: 0.1754
Epoch 2/100
4/4 [==============================] - 0s 11ms/step - loss: 0.3973 - output_variable1_loss: 0.2525 - output_variable2_loss: 0.1449 - val_loss: 0.2542 - val_output_variable1_loss: 0.1382 - val_output_variable2_loss: 0.1160
Epoch 3/100
4/4 [==============================] - 0s 11ms/step - loss: 0.2821 - output_variable1_loss: 0.1828 - output_variable2_loss: 0.0993 - val_loss: 0.1956 - val_output_variable1_loss: 0.1097 - val_output_variable2_loss: 0.0859
Epoch 4/100
4/4 [==============================] - 0s 11ms/step - loss: 0.2211 - output_variable1_loss: 0.1428 - output_variable2_loss: 0.0783 - val_loss: 0.1557 - val_output_variable1_loss: 0.0905 - val_output_variable2_loss: 0.0652
Epoch 5/100
4/4 [==============================] - 0s 11ms/step 

In [79]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.45  # For feature selection
columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[
        0]])) > THRESHOLD:  #or abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
independent_variables.remove(columns_to_predict[0])

if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(120, activation='tanh')(flatten_layer)
dense2 = layers.Dense(60, activation='tanh')(dense1)
dense3 = layers.Dense(24, activation='tanh')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(120, activation='tanh')(flatten_layer)
dense5 = layers.Dense(60, activation='tanh')(dense4)
dense6 = layers.Dense(24, activation='tanh')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_test)):
    current_batch = X_test[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    
print(independent_variables)

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 1s 55ms/step - loss: 0.7302 - output_variable1_loss: 0.4365 - output_variable2_loss: 0.2938 - val_loss: 0.3174 - val_output_variable1_loss: 0.1373 - val_output_variable2_loss: 0.1801
Epoch 2/100
4/4 [==============================] - 0s 11ms/step - loss: 0.3447 - output_variable1_loss: 0.1922 - output_variable2_loss: 0.1525 - val_loss: 0.2222 - val_output_variable1_loss: 0.1014 - val_output_variable2_loss: 0.1208
Epoch 3/100
4/4 [==============================] - 0s 11ms/step - loss: 0.2453 - output_variable1_loss: 0.1376 - output_variable2_loss: 0.1077 - val_loss: 0.1781 - val_output_variable1_loss: 0.0842 - val_output_variable2_loss: 0.0939
Epoch 4/100
4/4 [==============================] - 0s 11ms/step - loss: 0.1879 - output_variable1_loss: 0.1013 - output_variable2_loss: 0.0867 - val_loss: 0.1429 - val_output_variable1_loss: 0.0651 - val_output_variable2_loss: 0.0778
Epoch 5/100
4/4 [==============================] - 0s 10ms/step 

In [80]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.35  # For feature selection
columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[
        0]])) > THRESHOLD:  #or abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
independent_variables.remove(columns_to_predict[0])

if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(120, activation='tanh')(flatten_layer)
dense2 = layers.Dense(60, activation='tanh')(dense1)
dense3 = layers.Dense(24, activation='tanh')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(120, activation='tanh')(flatten_layer)
dense5 = layers.Dense(60, activation='tanh')(dense4)
dense6 = layers.Dense(24, activation='tanh')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_test)):
    current_batch = X_test[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

print(independent_variables)

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 1s 55ms/step - loss: 0.8126 - output_variable1_loss: 0.5301 - output_variable2_loss: 0.2825 - val_loss: 0.3554 - val_output_variable1_loss: 0.1891 - val_output_variable2_loss: 0.1664
Epoch 2/100
4/4 [==============================] - 0s 12ms/step - loss: 0.4214 - output_variable1_loss: 0.2615 - output_variable2_loss: 0.1600 - val_loss: 0.2412 - val_output_variable1_loss: 0.1060 - val_output_variable2_loss: 0.1352
Epoch 3/100
4/4 [==============================] - 0s 12ms/step - loss: 0.3006 - output_variable1_loss: 0.1751 - output_variable2_loss: 0.1255 - val_loss: 0.1787 - val_output_variable1_loss: 0.0731 - val_output_variable2_loss: 0.1056
Epoch 4/100
4/4 [==============================] - 0s 12ms/step - loss: 0.2227 - output_variable1_loss: 0.1228 - output_variable2_loss: 0.0999 - val_loss: 0.1584 - val_output_variable1_loss: 0.0656 - val_output_variable2_loss: 0.0928
Epoch 5/100
4/4 [==============================] - 0s 12ms/step 

In [81]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.3  # For feature selection
columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[
        0]])) > THRESHOLD:  #or abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
independent_variables.remove(columns_to_predict[0])

if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(120, activation='tanh')(flatten_layer)
dense2 = layers.Dense(60, activation='tanh')(dense1)
dense3 = layers.Dense(24, activation='tanh')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(120, activation='tanh')(flatten_layer)
dense5 = layers.Dense(60, activation='tanh')(dense4)
dense6 = layers.Dense(24, activation='tanh')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_test)):
    current_batch = X_test[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    
print(independent_variables)

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 1s 55ms/step - loss: 0.9749 - output_variable1_loss: 0.5073 - output_variable2_loss: 0.4676 - val_loss: 0.4420 - val_output_variable1_loss: 0.2137 - val_output_variable2_loss: 0.2283
Epoch 2/100
4/4 [==============================] - 0s 12ms/step - loss: 0.4601 - output_variable1_loss: 0.2377 - output_variable2_loss: 0.2224 - val_loss: 0.3295 - val_output_variable1_loss: 0.1555 - val_output_variable2_loss: 0.1740
Epoch 3/100
4/4 [==============================] - 0s 12ms/step - loss: 0.3322 - output_variable1_loss: 0.1743 - output_variable2_loss: 0.1578 - val_loss: 0.2310 - val_output_variable1_loss: 0.0928 - val_output_variable2_loss: 0.1382
Epoch 4/100
4/4 [==============================] - 0s 12ms/step - loss: 0.2616 - output_variable1_loss: 0.1337 - output_variable2_loss: 0.1279 - val_loss: 0.2017 - val_output_variable1_loss: 0.0796 - val_output_variable2_loss: 0.1221
Epoch 5/100
4/4 [==============================] - 0s 11ms/step 

In [82]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.25  # For feature selection
columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[
        0]])) > THRESHOLD:  #or abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
independent_variables.remove(columns_to_predict[0])

if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(120, activation='tanh')(flatten_layer)
dense2 = layers.Dense(60, activation='tanh')(dense1)
dense3 = layers.Dense(24, activation='tanh')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(120, activation='tanh')(flatten_layer)
dense5 = layers.Dense(60, activation='tanh')(dense4)
dense6 = layers.Dense(24, activation='tanh')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_test)):
    current_batch = X_test[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    
print(independent_variables)

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 1s 55ms/step - loss: 0.7326 - output_variable1_loss: 0.4071 - output_variable2_loss: 0.3255 - val_loss: 0.3333 - val_output_variable1_loss: 0.1736 - val_output_variable2_loss: 0.1597
Epoch 2/100
4/4 [==============================] - 0s 12ms/step - loss: 0.3231 - output_variable1_loss: 0.1818 - output_variable2_loss: 0.1412 - val_loss: 0.2029 - val_output_variable1_loss: 0.1004 - val_output_variable2_loss: 0.1025
Epoch 3/100
4/4 [==============================] - 0s 12ms/step - loss: 0.2109 - output_variable1_loss: 0.1205 - output_variable2_loss: 0.0904 - val_loss: 0.1775 - val_output_variable1_loss: 0.0887 - val_output_variable2_loss: 0.0887
Epoch 4/100
4/4 [==============================] - 0s 12ms/step - loss: 0.1774 - output_variable1_loss: 0.0976 - output_variable2_loss: 0.0798 - val_loss: 0.1400 - val_output_variable1_loss: 0.0668 - val_output_variable2_loss: 0.0732
Epoch 5/100
4/4 [==============================] - 0s 12ms/step 

In [83]:
print(RMSE_model_1)
print(RMSE_model_2)

[0.144636517867288, 0.1392831009893427, 0.13894739774996967, 0.13908194416689884, 0.15220280653519788, 0.1426656169725844, 0.1397258757359677, 0.1694994867562421, 0.18924019605113113, 0.15509481810843317, 0.13850912903489285, 0.1480090665194012, 0.15052127679045996]
[0.04885242890789899, 0.04448778497555304, 0.04477046082540157, 0.04402681795976877, 0.053467907282377256, 0.04982063058344877, 0.0395112503107179, 0.05081872308390406, 0.04446620475445795, 0.0557807932205749, 0.0451993515026592, 0.042871821464241865, 0.0405817916963572]
